In [ ]:
import tensorflow as tf
import os
'''
os.environ["CUDA_VISIBLE_DEVICES"]='0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [ ]:
#5NN + ReLu + Dropout

#setup place holder for data and label
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

#setup layers
keep_prob = tf.placeholder(tf.float32)
models = []
W1 = tf.Variable(tf.random_normal([784, 1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1)+B1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1,W2)+B2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512, 256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2,W3)+B3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))

models = tf.nn.softmax(tf.matmul(L3,W4)+B4)
for i in range(2):
    W1 = tf.Variable(tf.random_normal([784, 1024], stddev=0.01))
    B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
    L1 = tf.nn.relu(tf.matmul(X, W1)+B1)
    L1 = tf.nn.dropout(L1, keep_prob)

    W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
    B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
    L2 = tf.nn.relu(tf.matmul(L1,W2)+B2)
    L2 = tf.nn.dropout(L2, keep_prob)

    W3 = tf.Variable(tf.random_normal([512, 256], stddev=0.01))
    B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
    L3 = tf.nn.relu(tf.matmul(L2,W3)+B3)
    L3 = tf.nn.dropout(L3, keep_prob)

    W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
    B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))

    model = tf.nn.softmax(tf.matmul(L3,W4)+B4)
    models = tf.concat([models,model],-1)
      

print(models)

W = tf.Variable(tf.random_normal([30, 10], stddev=0.01))
B = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.relu(tf.matmul(models,W)+B)


#cost function and optimizer
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(tf.clip_by_value(model, 1e-10,1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#set up for minibatch
batch_size=100
total_batch= int(mnist.train.num_examples/batch_size) 

#set up for accuracy
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
#training
for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #_, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        #total_cost += cost_val
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.5})
        total_cost += cost_val
        
        #print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch), 
              #'Train Acc. =', sess.run(accuracy, feed_dict={X: mnist.train.images, Y: mnist.train.labels}))
        print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch), 'Train Acc. =', sess.run(accuracy, feed_dict={X: mnist.train.images, Y: mnist.train.labels, keep_prob: 1.0}))

#print("Training Done")
#print("Test Acc.=", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
        
print('Training Done!')
print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}))